# Quasi-periodic test asteroid

Run `rebound` to generate the data used to plot Figure 3.4 from Chapter 3 of Ryden, *Celestial and Stellar
Dynamics*, Cambridge University Press.

Compute eccentricity and semimajor axis versus time for a test-mass asteroid with initial orbital elements
 * $a_i=0.6944 a_\mathrm{jup}$
 * $e_i=0.2065$. 
 
Distance is in units of $a_\mathrm{jup}$; time is in units of $\mathcal{P}_\mathrm{jup}$. This plot is following 
plots in Murray & Dermott (1999), but we use the [rebound python/C package](https://rebound.readthedocs.io) to 
calcuate the orbits (Rein & Liu [2012, A&A, 537, 128](https://ui.adsabs.harvard.edu/abs/2012A%2526A...537A.128R)). 

For this and the calculation of a chaotic test asteroid in Figure 3.6, we set up a restricted 3-body problem of 
the Sun, Jupiter, and the test-mass asteroid, putting in the semimajor axis of Jupiter and its mass in M$_\mathrm{sun}$, but setting e=0 and i=0.

This script creates one ASCII 3-column text file:
 * `quasiAsteroid.txt` - time, a, and e

In [1]:
import rebound

import numpy as np

import warnings

## Setup the simulation

Simple system: Sun, Jupiter, and a test asteroid with
 * $a_i$ = 0.6944 a$_{jup}$
 * $e_e$ = 0.2065


In [2]:
sim = rebound.Simulation()

# sun

sim.add(m=1.)

# jupiter

aj = 5.2028870
ej = 0.04838624
ij = 1.30439695
mj = 9.547919e-4 # Msun

Pj = aj**1.5 # years

sim.add(m=mj,a=aj,e=0.0,inc=0.0) # ej,inc=ij)

# test asteroid - initial 

at_init = 0.6944*aj # quasiperiodic
et_init = 0.2065

sim.add(m=0.,a=at_init,e=et_init)

sim.move_to_com()

## Run the simulation

Run for 500 jupiter orbits and pick out 300 sequential orbits.  The dimensionless units of the
`rebound` simulation sets G=1, M=1sun, and so 1 years is 2$\pi$ in the simulation time units.

At the end, export the data to a 3-column ASCII text file with a comment header and 3 entrie
 * `t/PJ` - time in units of the period of Jupiter
 * `a/aj` - semimajor axis in units of a$_\textrm{Jup}$
 * `ecc` - eccentricity

In [3]:
# time units with G=1 are 1 year = 2pi

numPj = 500
numSteps = 1001
randSeed = 12345

tmax = numPj*Pj*2.0*np.pi

tjup = []
atmp= []
etmp = []
for t in np.linspace(0,tmax,numSteps):
    sim.rand_seed = randSeed
    sim.integrate(t)
    o = sim.orbits()
    tjup.append(numPj*t/tmax)
    atmp.append(o[1].a)
    etmp.append(o[1].e)
    
# extract the arrays and put the units in the form for the plot.

tPj = np.array(tjup)
at = np.array(atmp)/aj
et = np.array(etmp)

# restrict attention to a quasi-period series 300 Pjup long that starts at 200 Pjup

tmin = 200.
iorb = np.where(tPj >= tmin)[0]

tMax = numPj - tmin
aqp = at[iorb]
eqp = et[iorb]
tqp = tPj[iorb] - tmin

# open the data file

outFile = 'quasiAsteroid.txt'
f = open(outFile,'w')

# informational header

f.write('#\n')
f.write('# Quasi-periodic asteroid orbit integration for Fig 3.4 of Ryden Celestial and Stellar Dynamics\n')
f.write('#\n')
f.write(f'# tMax = {tMax} in units where 1 year = 2*pi\n')
f.write(f'# numPj = {numPj} number of Jupiter orbits\n')
f.write(f'# numSteps = {numSteps-1}\n')
f.write(f'# random seed = {randSeed}\n')
f.write(f'# a_init: {at_init:.4f} au\n')
f.write(f'# e_init: {et_init:.4f}\n')
f.write(f'# i_init: 0-deg\n')
f.write(f'#\n')

# data header

f.write(f't/PJ  a/aJ               ecc\n')

# data

for i in range(len(tqp)):
    f.write(f'{tqp[i]:5.1f} {aqp[i]:.16f} {eqp[i]:.16f}\n')

f.close()